In [25]:
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from pyxirr import xirr
pd.options.display.float_format = '{:,.2f}'.format


# Imputs

In [26]:
df_input=pd.read_csv('./input_fluxo_caixa_SQS312h604.csv',header=None,index_col=0)
df_input=df_input.rename(columns={1:'valor_variavel'})

In [27]:
valor_imovel=df_input.loc['valor_imovel','valor_variavel']
entrada_pp=df_input.loc['entrada_pp','valor_variavel']
entrada_por_valor=df_input.loc['entrada_por_valor','valor_variavel']
financiamento_0_ou_1=df_input.loc['financiamento_0_ou_1','valor_variavel']
area=df_input.loc['area','valor_variavel']

# Custos Iniciais

In [28]:
itbi_parcelado_0_ou_1=df_input.loc['itbi_parcelado_0_ou_1','valor_variavel']
custo_itbi=0.03*valor_imovel
custo_escritura=df_input.loc['custo_escritura','valor_variavel']
custo_registro=df_input.loc['custo_registro','valor_variavel']


# Cálculo financiamento

In [29]:
# Cálculo do juros em função do mês
def parcela(n):
    saldo_devedor=valor_finaciado-(n-1)*amortizacao
    juros=saldo_devedor*juros_nominal/12
    return amortizacao+juros+encargos_finaciamento


In [30]:
if financiamento_0_ou_1==1:
    n_parcelas=int(df_input.loc['n_parcelas','valor_variavel'])                         # 35 anos
    df=pd.DataFrame(columns=['parcela','saldo_devedor'],index=range(0,n_parcelas))
    juros_nominal=df_input.loc['juros_nominal','valor_variavel']                        # a.a    
    juros_tr=df_input.loc['tr','valor_variavel']                        # a.a
    juros_nominal=juros_nominal+juros_tr
    custo_avaliacao=df_input.loc['custo_avaliacao','valor_variavel']
    juros_efetivo=((1+juros_nominal/12)**(12)-1)            # Efeito anual dos juros sobre juros mensais.  

    # he usual way to test for a NaN is to see if it's equal to itself:
    if entrada_por_valor!=entrada_por_valor:
        # Não digitou o valor da entrada
        if entrada_pp!=entrada_pp:
            print('Digite o valor da entrada ou seu percentual.')
        else:
            valor_entrada=valor_imovel*entrada_pp
    else:
        valor_entrada=entrada_por_valor
    
    valor_finaciado=valor_imovel-valor_entrada
    encargos_finaciamento=df_input.loc['encargos_finaciamento','valor_variavel']
    amortizacao=valor_finaciado/n_parcelas
    saldo_devedor=valor_finaciado

else:
    amortizacao=0
    saldo_devedor=0
    n_parcelas=0   
    juros_nominal=0
    custo_avaliacao=0
    juros_efetivo=0  
    valor_finaciado=0
    encargos_finaciamento=0
    valor_entrada=valor_imovel
    df=pd.DataFrame()

In [31]:
total_custos_iniciais=valor_entrada+custo_escritura+custo_registro+custo_avaliacao

In [32]:
for i in range(0,max(24,n_parcelas+1)):             # Se não for financiado seta o df para 24 linhas inicialmente
    if i==0:
        df.loc[i,'saldo_devedor']=valor_finaciado-i*amortizacao
        df.loc[i,'parcela']=0
    else:

        df.loc[i,'saldo_devedor']=valor_finaciado-i*amortizacao
        df.loc[i,'parcela']=parcela(i)

In [33]:
# ITBI
if itbi_parcelado_0_ou_1==1:
    for i in range(0,len(df)):
        if i <10:
            df.loc[i,'itbi']=custo_itbi/10
        else:
            df.loc[i,'itbi']=0
else:
    df.loc[0,'itbi']=custo_itbi
    for i in range(1,len(df)):
        df.loc[0,'itbi']=0

# Custos fixos

In [34]:
condominio=df_input.loc['condominio','valor_variavel']
iptu_am=df_input.loc['iptu_aa','valor_variavel']/12
luz=df_input.loc['luz','valor_variavel']

total_custos_fixos=condominio+iptu_am+luz

In [35]:
for i in range(0,len(df)):
    df.loc[i,'custos_fixos']=total_custos_fixos
    if i==0:
        df.loc[i,'custos_iniciais']=total_custos_iniciais
    else:
        df.loc[i,'custos_iniciais']=0

# Custos com reforma

In [36]:
obra=df_input.loc['obra','valor_variavel']
prazo_obra=df_input.loc['prazo_obra','valor_variavel']                # Prazo da obra em mesês

def reforma(i, n,total):
    if i==0:
        return 0
    elif i<=n:
        return total/n
    else:
        return 0

for i in range(0,len(df)):
    df.loc[i,'reforma']=reforma(i,prazo_obra,obra)

# Cenários de venda

In [37]:
valor_venda=df_input.loc['valor_venda','valor_variavel']
prazo_venda=int(prazo_obra+df_input.loc['prazo_venda_pos_obra','valor_variavel'])                # Prazo da obra em mesês + venda
comissao=df_input.loc['comissao','valor_variavel']

In [38]:
def venda(i, n,total):
    if i==n:
        print(f'Comissão: {comissao*valor_venda}')
        return -total*(1-comissao)
    else:
        return 0

for i in range(0,len(df)):
    df.loc[i,'venda']=venda(i,prazo_venda,valor_venda)
df['total']=df.sum(axis=1)
df['total']=df['total']-df['saldo_devedor']

# quitação do saldo devedor e anulação das despesas à partir da venda
for i in range(0,len(df)):
    if i>prazo_venda:
        df.loc[i,'total']=0
    elif i==prazo_venda:
        df.loc[i,'total']=df.loc[i,'total']+df.loc[i,'saldo_devedor']

Comissão: 13500.0


In [39]:
delta = relativedelta(months=1)
delta_d = timedelta(days=1)
start_date = datetime.today().date()+15*delta_d

for i in range(0, len(df)):
    if i==0:
        df.loc[i,'data']=start_date
    else:
        df.loc[i,'data']=start_date+i*delta

# Filtra apenas até a célula de venda
df=df[df['total']!=0]
df

,parcela,saldo_devedor,itbi,custos_fixos,custos_iniciais,reforma,venda,total,data
0,0,"561,000.00","2,115.00",885.67,"148,900.00",0.00,0.00,"151,900.67",2022-09-07
1,"6,276.29","559,664.29","2,115.00",885.67,0.00,"13,333.33",0.00,"22,610.29",2022-10-07
2,"6,264.84","558,328.57","2,115.00",885.67,0.00,"13,333.33",0.00,"22,598.84",2022-11-07
3,"6,253.38","556,992.86","2,115.00",885.67,0.00,"13,333.33",0.00,"22,587.38",2022-12-07
4,"6,241.93","555,657.14","2,115.00",885.67,0.00,"13,333.33",0.00,"22,575.93",2023-01-07
5,"6,230.47","554,321.43","2,115.00",885.67,0.00,"13,333.33",0.00,"22,564.47",2023-02-07
6,"6,219.02","552,985.71","2,115.00",885.67,0.00,"13,333.33",0.00,"22,553.02",2023-03-07
7,"6,207.57","551,650.00","2,115.00",885.67,0.00,0.00,0.00,"9,208.23",2023-04-07
8,"6,196.11","550,314.29","2,115.00",885.67,0.00,0.00,0.00,"9,196.78",2023-05-07
9,"6,184.66","548,978.57","2,115.00",885.67,0.00,0.00,"-886,500.00","-328,336.10",2023-06-07


In [40]:
# Importa output da BcB_forcast (os dois diretórios no mesmo caminho)
df_juros_real=pd.read_csv(r'../BcB_Forcast/output/num_indice_selic_ipca_diferenca.csv', index_col=0)
df_juros_real

,num_indice_selic,num_indice_ipca,dif_num_indice
2022-08-23,100.00,100.00,100.00
2022-08-24,100.04,99.99,100.04
2022-08-25,100.07,99.98,100.09
2022-08-26,100.11,99.97,100.13
2022-08-27,100.14,99.96,100.18
...,...,...,...
2024-08-10,125.41,109.89,115.52
2024-08-11,125.44,109.89,115.55
2024-08-12,125.47,109.90,115.57
2024-08-13,125.50,109.90,115.59


In [41]:
def busca_num_indice_selic(data):
    #converte data em string
    data=datetime.strftime(data,'%Y-%m-%d')    
    return df_juros_real.loc[data,'num_indice_selic']
df['num_selic']=df.apply(lambda x: busca_num_indice_selic(x.data),axis=1)

def corrige_selic(total,num_selic):
    # Número índice de referência (data da venda)
    num_indice_ref=df.loc[df.index[-1],'num_selic']
    total=total*num_indice_ref/num_selic
    return total
df['total_selic']=df.apply(lambda x: corrige_selic(x.total,x.num_selic),axis=1)

df

,parcela,saldo_devedor,itbi,custos_fixos,custos_iniciais,reforma,venda,total,data,num_selic,total_selic
0,0,"561,000.00","2,115.00",885.67,"148,900.00",0.00,0.00,"151,900.67",2022-09-07,100.53,"167,267.52"
1,"6,276.29","559,664.29","2,115.00",885.67,0.00,"13,333.33",0.00,"22,610.29",2022-10-07,101.60,"24,635.42"
2,"6,264.84","558,328.57","2,115.00",885.67,0.00,"13,333.33",0.00,"22,598.84",2022-11-07,102.72,"24,354.47"
3,"6,253.38","556,992.86","2,115.00",885.67,0.00,"13,333.33",0.00,"22,587.38",2022-12-07,103.81,"24,086.53"
4,"6,241.93","555,657.14","2,115.00",885.67,0.00,"13,333.33",0.00,"22,575.93",2023-01-07,104.95,"23,812.82"
5,"6,230.47","554,321.43","2,115.00",885.67,0.00,"13,333.33",0.00,"22,564.47",2023-02-07,106.11,"23,540.55"
6,"6,219.02","552,985.71","2,115.00",885.67,0.00,"13,333.33",0.00,"22,553.02",2023-03-07,107.16,"23,298.05"
7,"6,207.57","551,650.00","2,115.00",885.67,0.00,0.00,0.00,"9,208.23",2023-04-07,108.34,"9,408.82"
8,"6,196.11","550,314.29","2,115.00",885.67,0.00,0.00,0.00,"9,196.78",2023-05-07,109.50,"9,297.57"
9,"6,184.66","548,978.57","2,115.00",885.67,0.00,0.00,"-886,500.00","-328,336.10",2023-06-07,110.70,"-328,336.10"


In [42]:
# calculo da IRR
def cal_irr(x):
    
    dates=list(x['data'])
    amounts=list(x['total'])
    t_aa=xirr(dates, amounts)
    t_am=(1+t_aa)**(1/12)-1
    print('TII a.a.(%): ',t_aa*100)
    print('TII a.m.(%): ',t_am*100)
    print('Lucro: ',-x['total'].sum())

    amounts_selic=list(x['total_selic'])
    t_aa=xirr(dates, amounts_selic)
    t_am=(1+t_aa)**(1/12)-1
    print('TII a.a. acima da selic (%): ',t_aa*100)
    print('TII a.m. acima da selic (%): ',t_am*100)
    print('Lucro: ',-x['total_selic'].sum())
    
    # return (t_aa*100,t_am*100,-x['total'].sum())

In [43]:
cal_irr(df)

TII a.a.(%):  12.949861042284494
TII a.m.(%):  1.0199482613130861
Lucro:  22540.493333333405
TII a.a. acima da selic (%):  -0.7049454134163002
TII a.m. acima da selic (%):  -0.058936116932162275
Lucro:  -1365.646960598824


In [44]:
valor_m2_aquisicao=valor_imovel/area
print(f'm2 aquisição: {valor_m2_aquisicao}')
valor_m2_venda=valor_venda/area
print(f'm2 reforma: {obra/area}')
print(f'm2 venda: {valor_m2_venda}')


m2 aquisição: 10143.884892086331
m2 reforma: 1151.0791366906474
m2 venda: 12949.640287769784


In [45]:
11*66+70

796

In [46]:
### projeções de inflação e crescimento econômico para evolução dos preços de venda